In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Trabalho Pratico 2
#### Analise consumo e produção de matrizes energéticas brasileiras comparadas com o mundo.
*Vitor Hugo - 3049*  
*Jose Gregorio - 3046*

## Introdução
##### O dado Trabalho foi realizado com o intuito de aplicar os conhecimentos vistos nas aulas e encontros remotos da materia de ciencia de dados.
##### O trabalho foi divido em 4 parte escolha dos dados, preparação dos dados, análise exploratória e análise preditiva estás estarão explícitas no documento para melhor entendimento do fluxo de desenvolvimento do trabalho.

## Escolha e motivação

##### A motivação encontrada para realização do trabalho além de aplicarmos de forma realística os conhecimentos visto em aula queríamos a possibilidade de realizar análise sobre assuntos que tangem a relação do brasil com o meio ambiente, assim escolhemos falar sobre dados energéticos e seus consumos ao longo dos anos, que é um retrato nítido de como vem sendo levado questões ambientais no país.  E além do contexto brasileiro queríamos comparar o brasil com outros países do mundo, para melhor entendimento de como estão as matrizes energéticas ao longo do globo.
##### Para isso os dados escolhidos para análise foram encontrados na IEA(Internetional Energy Agency) no seguinte [link](https://www.iea.org/subscribe-to-data-services/world-energy-balances-and-statistics).Na tabela encontrada no link temos dados das matrizes energeticas de 44 paises diferentes dispostos durante os anos de 1971 a 2018.
##### Dessa forma, essa tabela nos permite um panorama geral dos meios de produção energético de cada país e o fluxo tomado por essa energia produzida. Assim, através desses dados podemos extrair muitos conhecimentos sobre como os países estão lidando com suas matrizes energéticas ao longo dos anos é quais setores mais consomem essa energia.

## Preparação dos dados

##### O primeiro passo da preparação dos dados foi o entendimento mais profundo da tabela e o significado de cada label nela presente. Para isso criamos uma tabela de índices que explica melhor como foi agrupado os dados na tabela e o significado de cada label da tabela usada.
##### Devido a extensão do arquivos criamos este em outro local para ficar mais facil seu entendimento [link](indexes_IEA.md).


## Selecionando tabela bruta

In [ ]:
xls = pd.ExcelFile('../data/WorldEnergyBalancesHighlights_final.xlsx')

In [ ]:
df = pd.read_excel(xls, 'TimeSeries_1971-2019')
df.head(5)

### Limpando e tratando dados
- O primeiro problema que encontramos ao analisar a tabela bruta mostrada acima foi a falha ao atribuir o header, de forma que os headers estão sendo reconhecidos como Unnamed. Para resolver esse problema ao iniciar a tabela colocamos como padrão a linha de numero 1 como header, assim os headers corretos são reconhecidos.

In [ ]:
df = pd.read_excel(xls, 'TimeSeries_1971-2019',header=1)
df.head(5)

- Outro problema encontrado na tabela foi acriação dos indexes automaticos que não condiziam com os indexes que gostariamos Country, Product e Flow. Assim para solucionar esse problema aplicamos as colunas 0,1 e 2 como as colunas indexes.

In [ ]:
df = pd.read_excel(xls, 'TimeSeries_1971-2019',header=1,index_col=[0,1,2])
df.head(5)

- Outros 2 problemas encontrados na tabela foi a leitura incorreta de colunas que não haviam dados com as "Unnamed: 55" a "Unnamed: 64", e as colunas com valores de repetidos de outras colunas como e o caso das colunas NoCountry, NoProduct, NoFlow. Para esses problemas como estavamos usando uma tabela do exel a solução foi basicamente selecionar os ranges dos nomes das colunas que queriamos usar, assim descartando as outras com erros.

In [ ]:
df = pd.read_excel(xls, 'TimeSeries_1971-2019',header=1,index_col=[0,1,2],usecols="A:C,G:BC")
df.head(5)

In [ ]:
## Lendo dados do Brasil
br = df.loc[('Brazil')]
br.head(5)

- Ao lermos os dados do Brasil é perceptivel que não há dados entre os anos de 1971-1989 e 2019(devido estarem sendo apurado ainda). Assim, foi removido as colunas referente aos anos de 1971-1989, 2019, pois não seria interessante em nossas analises já que o Brasil é o foco principal de analise. Novamente, utilizamos do artificio de podermos selecionar o range dos nomes das colunas que nos mais interessa.

In [ ]:
df = pd.read_excel(xls, 'TimeSeries_1971-2019',header=1,index_col=[0,1,2],usecols="A:C,Z:BB")
df.head(5)

In [ ]:
df.dtypes

- Percebemos que os tipos de dados das colunas estevão como objetos apesar de ser esperados serem do tipo float. Assim ao estudarmos a fundo a planilha e abrir está em um leitor de arquivos xlsx, encontramos na pagina definitions a existencia de dois tipos de valores os  '..' : Não há dados e o 'c' : confidencial. Assim percebemos que haveria a necessidade de transformar esses valores em valores Nan para que houve-se a possibilidade de tratar as colunas como float.

In [ ]:
df = pd.read_excel(xls, 'TimeSeries_1971-2019',header=1,index_col=[0,1,2],usecols="A:C,Z:BB",na_values=['..','c'])
df.dtypes

## Análise exploratória
##### Na analise exploratoria procuramos entender melhor a correlação possiveis entre os dados, além das informações e conhecimentos que poderiamos extrair com bases nas correlações e analises.

### Preparação
##### Nosso primeiro objetivo foi ter uma lista explicita com os paises sem a interferencia dos agrupamentos regionais para isso pegamos o indexe dos paises e limpamos manualmente tudo aquilo que erá um agrupamento.

In [ ]:
df = pd.read_excel(xls, 'TimeSeries_1971-2019',header=1,index_col=[0,1,2],usecols="A:C,Z:BB",na_values=['..','c'])
indexes:pd.MultiIndex  = df.index
pd.unique(indexes.get_level_values('Country'))

In [ ]:
countries = [
"Australia",
"Austria",
"Belgium",
"Canada",
"Chile",
"Czech Republic",
"Denmark",
"Estonia",
"Finland",
"France",
"Germany",
"Greece",
"Hungary",
"Iceland",
"Ireland",
"Israel",
"Italy",
"Japan",
"Korea",
"Latvia",
"Lithuania",
"Luxembourg",
"Mexico",
"Netherlands",
"New Zealand",
"Norway",
"Poland",
"Portugal",
"Slovak Republic",
"Slovenia",
"Spain",
"Sweden",
"Switzerland",
"Turkey",
"United Kingdom",
"United States",
"Brazil",
"People's Republic of China",
"India",
"Indonesia",
"Morocco",
"Singapore",
"South Africa",
"Thailand"]

##### Também achamos interessante agrupar esses paises em continentes, uma vez que na etapa de gerar perguntas sobre os dados criamos perguntas relacionadas a comparações entre continentes.

In [ ]:
asia = [
"Japan",
"Korea",
"People's Republic of China",
"India",
"Indonesia",
"Singapore",
"Thailand"
]

oceania= [
"Australia",
"New Zealand"]

america= [
"Canada",
"Chile",
"Mexico",
"United States",
"Brazil",
]

europa= [
"Austria",
"Belgium",
"Czech Republic",
"Denmark",
"Estonia",
"Finland",
"France",
"Germany",
"Greece",
"Hungary",
"Iceland",
"Ireland",
"Israel",
"Italy",
"Latvia",
"Lithuania",
"Luxembourg",
"Netherlands",
"New Zealand",
"Norway",
"Poland",
"Portugal",
"Slovak Republic",
"Slovenia",
"Spain",
"Sweden",
"Switzerland",
"Turkey",
"United Kingdom",]

##### Também foi relevante a criação de uma tabela só com os dados do Brasil que é o nosso foco de análise e do mundo para fácil usabilidade e comparação.Além disso geramos csv's dos mesmo para podermos termos uma melhor dimensão de alguns dados e características de possíveis análises.

In [ ]:
br = df.loc[('Brazil')]
world = df.loc[('World')]

world.to_csv('world.csv')
br.to_csv('br.csv')

##### Por fim a ultima preparação que fizemos nos dados foi criar uma lista com todos os anos para facilitar a criação de tabelas e talvez futuras analises.

In [ ]:
years = df.transpose().index.tolist()
def commanPlot(obj,list_year=years):
    obj.plot(figsize=(15,10))
    plt.xticks(list_year)
    plt.xticks(rotation=90)
    plt.show()

### Analise exploratória

#### 1) No grafo a seguir realizamos comparações entre os 4 consumidores finais das matrizes energeticas brasileiras, sendo a linha de azul a soma total destas.

In [ ]:
total:pd.DataFrame = br.loc[('Total')].iloc[6:12].transpose()
commanPlot(total)

##### Conclusões: É perceptivel que os maiores consumidores no brasil são as areas de transporte e da industria, o que já era de se esperar devido suas grandes demandas energeticas, todavia o que nos surpreendeu foi o fato da industria agricola inclusa no(Other final consumption) se a que menos gasta enrgia entre as demais.

#### 2) No grafic a seguir realizamos comparações entre as 3 formas de produção de energia, sendo a linha vermelha o somatoria das demais.

In [ ]:
#produção energetica no brasil
eletric = br.xs('Electricity output (GWh)',level='Flow').transpose()
commanPlot(eletric)

##### Conclusões: É nitido a lacuna de mais de 10000 Gwh de diferenca entre a produção de energia Fossil em comparação com a renovavel. Todavia ao realizar esse grafico e pensar sobre ele nos veio a questão se essa lacuna também se reflete no consumo, se realmente toda essa enegia renovavel é a mais consumida pelos setores de consumo.

##### Outra questãos que nós intrigou foi quanto ao pico de produção de energia fossil entre os anos de 2011 a 2016, estipulamos que talvés  este fato estaja corelacionado com os anos de crise hidrica no Brasil, mas bem provavel que haja outros fatores por tras como o inicio da exportação do petrolio porduzido no pré sal.

#### 3) Nos graficos a seguir vemos as comparações do consumos total de energia ao longo dos anos e mais detalhadamente no ano de 2018 

In [ ]:
#consumo de energias renovaveis no brasil
total_br_prod_renew = br.loc[('Renewables and waste')].iloc[6:12].transpose()
total_br_prod_renew['Total Renovavel'] = total_br_prod_renew['Total final consumption (ktoe)']

#consumo de derivados do petroleo
total_br_prod_fossiel = br.loc[('Oil products')].iloc[6:12].transpose()
total_br_prod_fossiel['Total Derivados petroleo'] = total_br_prod_fossiel['Total final consumption (ktoe)']

#consumo de gas natural no brasil
total_br_prod_natural = br.loc[('Natural gas')].iloc[6:12].transpose()
total_br_prod_natural['Total Gás Natural'] = total_br_prod_natural['Total final consumption (ktoe)']

#consumo de energia nuclear
total_br_prod_nuclear = br.loc[('Nuclear')].iloc[6:12].transpose()
total_br_prod_nuclear['Total Nuclear'] = total_br_prod_nuclear['Total final consumption (ktoe)']

#consumo de carvão
total_br_prod_coal = br.loc[('Coal, peat and oil shale')].iloc[6:12].transpose()
total_br_prod_coal['Total Carvão'] = total_br_prod_coal['Total final consumption (ktoe)']

In [ ]:
# Comparação do total do consumo ao longo dos anos de cada tipo de fonte energetica
teste = pd.concat([total_br_prod_fossiel['Total Derivados petroleo'],total_br_prod_coal['Total Carvão'],total_br_prod_natural['Total Gás Natural'],total_br_prod_nuclear['Total Nuclear'],total_br_prod_renew['Total Renovavel']],axis=1)
teste.plot(kind='bar',figsize=(20,10))
plt.legend(['Derivados do petroleo','Carvão','Gás Natural','Nuclear','Renovavel'])

In [ ]:
# Comparação do total do consumo no ano de 2018 de cada tipo de fonte energetica
teste.loc[(2018)].plot(kind='bar',figsize=(20,10))

##### Conclusões: Como mostrado pelos graficos percebemos que os consumos totais são liderados pela energia renovavel e principalmente pelos produtos derivados do petroleo o que é estranho, pois apesar da grande produção de energia limpa não a usamos em todo seu pontencia. Outra analise pode ser feita quanto aos três menos usados gaś natural, nuclear e carvão, percebemos que o carvão vem perdendo terreno ao longo dos anos e para o cinsumo de gás natuaral e já a nuclear é tão pouco seu consumo que nem aparece nos graficos. 

#### 4) O grafico a seguir compara as diversas fontes de consumos especificamente das energias renovaveis

In [ ]:
commanPlot(total_br_prod_renew)

##### Conclusões: Uma conclusão interessante retirada e o fato de as industrias terem um alto consumo de enrgias renovaveis o que mostra que muitas destas devem possuir uma possivel baixa emissão de carbono, outro ponto possitivo e o aumento do consumos dessas energias por parte da indutria do transporte que ao longo do tempo vem aumentando grandemente seu consumo. Por outro lado algo que nos instigou foi sobre o consumo residencial, esperavamos que este fosse maior devido ao cresimento da população ao longo dos anos e também devido a maior possibilidade de aquisição de produtos eletronicos por individuo. 

#### 5) O grafico a seguir compara as diversas fontes de consumos especificamente dos produtos derivados do petroleo

In [ ]:
commanPlot(total_br_prod_fossiel)

##### Conclusões: Como esperado a industria do transporte e a maior consumidora desse tipo de energia , vale resaltar que no ano de 2018 a diferença entre este e o segundo colocado e de significantes 30000 ktoe aproximadamente. Outra analise interessante e quanto a industria agricula que apesar de seu baixo consumo energetico como visto nos graficos anteriores está e a segunda maior consumidora de energia fóssil. Por fim, o consumo das industrias, residencias e comercio são  os que possue menores, e bem menores também quando se comprado ao seus consumos de energia renovavel. Uma conclusão parcial que podemos chegar é que a produção de enrgia fóssil no Brasil ainda só possue seu poder devido ao consumo da industria de transporte que chega a ser maior que o consumos das industrias com energias renovaveis. 

#### 6) Os graficos a seguir mostrará o consumo residencia entre as diferente fontes de produção de energia

In [ ]:
#consumo residencial no brasil
residential = br.xs('Residential (ktoe)',level='Flow').iloc[0:6].transpose()
commanPlot(residential)

In [ ]:
residential.loc[(2018)].plot(kind='pie',autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)

##### Conclusões: O consumo energético de residência se baseia nos dois grandes produtores energético fóssil e renovável, enquanto as outras fontes representam, praticamente 0% como visto no gráfico de pizza. Algo que notamos de estranho foi o comportamento da energia renovável ao longo dos anos uma vez que em 1990 esta possui uma altíssima taxa de uso e por algum motivo do qual não conseguimos constatar nos ano de 1992 esta teve um brusca queda que se repetiu de forma mais sutil a partir de 2006 a ponto de em 2013 a energia fóssil superar seu consumo.

#### 7) Os graficos a seguir mostrará o consumo do mundo dividio entre alguns setores, incluindo a soma total de consumo

In [ ]:
total_wrld:pd.DataFrame = world.loc[('Total')].iloc[6:12].transpose()

#consumo total no mundo inteiro
total_wrld.plot(figsize=(15,10))
plt.xticks(years)
plt.xticks(rotation=90)
plt.yscale('log')
plt.show()

##### Conclusões: 

#### 9)

In [ ]:
#Comparação de produção de energia mundial
eletric_wrld = world.xs('Electricity output (GWh)',level='Flow').transpose()
commanPlot(eletric_wrld)

##### Conclusões: 

#### 8)

In [ ]:
#Comparaçao da producao de energia no brasil desde 1990
eletricity_output = df.xs('Electricity output (GWh)',level='Flow')
eletricity_output_nototal = eletricity_output[eletricity_output.index.get_level_values('Product') != 'Total']
eletricity_output_br  = eletricity_output_nototal.loc[('Brazil')]
eletricity_output_br.transpose().sum().plot.bar()

##### Conclusões: 

#### 9)

In [ ]:
#maiores produtores de energias renovaveis desde 1990
eletricity_output_renew = eletricity_output.xs('Renewable sources',level='Product')
eletricity_output_renew.filter(items=countries, axis=0).transpose().sum().sort_values(ascending = False).plot.bar(figsize=(15,10))

##### Conclusões: 

#### 10)

In [ ]:
#maiores produtores em 2018
eletricity_output_renew.filter(items=countries, axis=0)[2018].transpose().sort_values(ascending = False).plot.bar(figsize=(15,10))

##### Conclusões: 

#### 11)

In [ ]:
#maiores consumidores de energia no total
all_total = df.xs('Total', level = 'Product')
all_total_consuption = all_total.xs('Total final consumption (ktoe)', level = 'Flow')
all_total_consuption.filter(items=countries, axis=0).transpose().sum().sort_values(ascending = False).plot.bar(figsize=(15,10))

##### Conclusões: 

#### 12)

In [ ]:
#maiores consumidores em 2018
all_total_consuption.filter(items=countries, axis=0)[2018].transpose().sort_values(ascending = False).plot.bar(figsize=(15,10))

##### Conclusões:

#### 13)

In [ ]:
#crescimento do uso de energias não renovaveis em sob as energias renovaveis
all_consuption = df.xs('Total final consumption (ktoe)', level = 'Flow')
all_total_cons = all_consuption.xs('Total', level = 'Product')
all_renew_cons = all_consuption.xs('Renewables and waste', level = 'Product')
## all_total_renew_ratio é igual a todo consumo menos o consumo renovavel, divido pelo consumo renovavel, ou seja, todo consumo não renovavel divido pelo consumo renovavel, é a grandeza de vezes a qual o consumo nao renovavel é maior que o consumo renovavel.
all_total_renew_ratio = (all_total_cons - all_renew_cons) / all_renew_cons
commanPlot(all_total_renew_ratio.loc[('World')])

##### Conclusões:

#### 14)

In [ ]:
# africa é o unico continente que utiliza mais energia renovavel que não renovavel, mesmo que entre os dados existam varios paises
renovaveis2018 = all_total_renew_ratio[all_total_renew_ratio[2018] <= 1]
renovaveis2018

##### Conclusões:

#### 15)


In [ ]:
#quais setores a energia renovavel cresceu
world.loc[('Renewables and waste')].iloc[7:].transpose().plot(figsize=(15,10))
plt.xticks(years)
plt.xticks(rotation=90)
plt.yscale('log')
plt.show()

##### Conclusões:

#### 16)

In [ ]:
#crescimento da produção total ao longo dos anos por regioẽs
regions_prod = eletricity_output.xs('Total', level = 'Product')
europe = regions_prod.filter(items=europa, axis=0).sum()
asia_prod = regions_prod.filter(items=asia, axis=0).sum()
america_prod = regions_prod.filter(items=america, axis=0).sum()
oceania_prod = regions_prod.filter(items=oceania, axis=0).sum()
africa_prod = eletricity_output.loc[('Africa', 'Total')]
plt.figure(figsize=(15,10))
europe.plot()
asia_prod.plot()
america_prod.plot()
oceania_prod.plot()
africa_prod.plot()
plt.xticks(years)
plt.xticks(rotation=90)
plt.yscale('log')
plt.legend(['Europa','Asia', 'America', 'Oceania', 'Africa'])
plt.show()

##### Conclusões:

#### 17)

In [ ]:
#crescimento da produção de energia renovavel ao longo dos anos por regioẽs
regions_renew = eletricity_output.xs('Renewable sources', level = 'Product')
europe_renew = regions_renew.filter(items=europa, axis=0).sum()
asia_renew = regions_renew.filter(items=asia, axis=0).sum()
america_renew = regions_renew.filter(items=america, axis=0).sum()
oceania_renew = regions_renew.filter(items=oceania, axis=0).sum()
africa_renew = eletricity_output.loc[('Africa', 'Renewable sources')]
plt.figure(figsize=(15,10))
europe_renew.plot()
asia_renew.plot()
america_renew.plot()
oceania_renew.plot()
africa_renew.plot()
plt.legend(['Europa','Asia', 'America', 'Oceania', 'Africa'])
plt.xticks(years)
plt.xticks(rotation=90)
plt.show()

##### Conclusões:
